In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from nets import *
from cfgs import *
from data import *
from trainer import *
from payment_utils import get_payment


In [ ]:
%matplotlib inline
save_plot = False
plt.rcParams.update({'font.size': 12, 'axes.labelsize': 'x-large'})
# グリッド解像度を向上（201 → 401）
D = 401

x = np.linspace(0, 2.5, D)
X_tst = np.stack([v.flatten() for v in np.meshgrid(x,x)], axis = -1)
print(X_tst.shape)

cfg = additive_1x2_gamma_100_1_config.cfg
cfg.test.batch_size = D
cfg.test.num_batches = int(X_tst.shape[0]/cfg.test.batch_size)
cfg.test.restore_iter = 400000  # より詳細な訓練モデルを使用
cfg.test.save_output = True


In [ ]:
Net = additive_net.Net
Generator = gamma_100_1_generator.Generator
Trainer = trainer.Trainer


In [ ]:
net = Net(cfg, "test")
generator = Generator(cfg, 'test', X_tst)
m = Trainer(cfg, "test", net)
m.test(generator)


In [ ]:
alloc = np.load(os.path.join(cfg.dir_name, "alloc_tst_" + str(cfg.test.restore_iter) + ".npy")).reshape(D,D,2)
pay = np.load(os.path.join(cfg.dir_name, "pay_tst_" + str(cfg.test.restore_iter) + ".npy")).reshape(D,D)


In [ ]:
plt.rcParams.update({'font.size': 12, 'axes.labelsize': 'x-large'})
# プロット解像度を向上（figsizeを大きく、DPIを上げる）
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (12, 10), dpi=150)

# より滑らかな補間を使用
img = ax.imshow(alloc[::-1, :, 0], extent=[0,2.5,0,2.5], vmin = 0.0, vmax=1.0, 
                cmap = 'YlOrRd', interpolation='bilinear', aspect='auto')
                    
ax.set_xlabel(r'$v_1$', fontsize=14)
ax.set_ylabel(r'$v_2$', fontsize=14)
plt.title('Prob. of allocating item 1', fontsize=16)
_ = plt.colorbar(img, fraction=0.046, pad=0.04)

if save_plot:
    fig.set_size_inches(8, 6)
    plt.savefig(os.path.join(cfg.dir_name, 'alloc1.pdf'), bbox_inches = 'tight', 
                pad_inches = 0.05, dpi=300)  # 高解像度で保存


In [ ]:
plt.rcParams.update({'font.size': 12, 'axes.labelsize': 'x-large'})
# プロット解像度を向上（figsizeを大きく、DPIを上げる）
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (12, 10), dpi=150)

# より滑らかな補間を使用
img = ax.imshow(alloc[::-1, :, 1], extent=[0,2.5,0,2.5], vmin = 0.0, vmax=1.0, 
                cmap = 'YlOrRd', interpolation='bilinear', aspect='auto')
              
ax.set_xlabel(r'$v_1$', fontsize=14)
ax.set_ylabel(r'$v_2$', fontsize=14)
plt.title('Prob. of allocating item 2', fontsize=16)
_ = plt.colorbar(img, fraction=0.046, pad=0.04)

if save_plot:
    fig.set_size_inches(8, 6)
    plt.savefig(os.path.join(cfg.dir_name, 'alloc2.pdf'), bbox_inches = 'tight', 
                pad_inches = 0.05, dpi=300)  # 高解像度で保存


In [ ]:
# インタラクティブなプロット（plotly使用）- マウスホバーで座標と値を表示
x_max = x[-1]  # Cell 1で定義されたxの最大値を使用

# データを準備
pay_display = pay  # データは反転しない
x_coords = np.linspace(0, x_max, D)
y_coords = np.linspace(0, x_max, D)

# plotlyのヒートマップを作成（より高解像度、より大きなサイズ）
fig = go.Figure(data=go.Heatmap(
    z=pay_display,
    x=x_coords,
    y=y_coords,
    colorscale='YlOrRd',
    colorbar=dict(title=dict(text="Payment", font=dict(size=14))),
    hovertemplate='v₁=%{x:.4f}<br>v₂=%{y:.4f}<br>Payment=%{z:.8f}<extra></extra>',
    name=''
))

fig.update_layout(
    title='Payment (Interactive - Hover to see values)',
    xaxis_title='v₁',
    yaxis_title='v₂',
    width=1200,  # 800から1200に増加
    height=1000,  # 600から1000に増加
    font=dict(size=12)
)

# HTMLファイルとして保存して表示（最も確実な方法）
html_file = os.path.join(cfg.dir_name, 'pay_interactive.html')
fig.write_html(html_file)
print(f"インタラクティブなプロットを保存しました: {html_file}")
print("ブラウザで開くと、マウスホバーで座標と値を表示できます。")

# Jupyter Notebook内で表示を試みる
try:
    from IPython.display import HTML, IFrame
    display(HTML(f'<iframe src="{html_file}" width="1250" height="1050"></iframe>'))
except:
    try:
        from IPython.display import display, HTML
        display(HTML(f'<a href="{html_file}" target="_blank">インタラクティブなプロットを開く</a>'))
    except:
        pass

if save_plot:
    try:
        fig.write_image(os.path.join(cfg.dir_name, 'pay.pdf'))
    except:
        pass  # write_imageが使えない場合はスキップ


In [ ]:
# 使用例
idx1, idx2, actual_v1, actual_v2, payment_val = get_payment(x, pay, 0, 1.90)
print(f"v1={actual_v1:.4f}, v2={actual_v2:.4f}: payment={payment_val:.6f}")
